In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("C:\\Users\\PRATEEK\\Desktop\\Prateek\\Data\\phishing_dataset.csv")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 822010 entries, 0 to 822009
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     822010 non-null  object
 1   status  822010 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 12.5+ MB


In [6]:
df.describe()

,status
count,822010.000000
mean,0.519492
std,0.499620
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [7]:
df.shape

(822010, 2)

In [11]:
pd.set_option('display.max_colwidth', None)
df.sample(10)

,url,status
402491,movies.yahoo.com/movie/contributor/1800017398,1
58717,https://www.rakutenn.c0.jp3.startfrom5512.xyz/,0
751839,887ec115da.000webhostapp.com,0
69282,https://tgv-intl.com/wp-content/themes/13/vbv/,0
616026,sfgate.com/sports/warriors/,1
540235,hookerdunham.org/,1
438087,twitter.com/simon_brault,1
622585,spokeo.com/Alonzo+Clark,1
9333,enders11.5gbfree.com,0
35933,https://westsideautosale.com/whb7j/office/index.php,0


In [ ]:
df.sum().isna()